# Playground for Experiments

We try to solve discrimination on the <a href=https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29>German Credit Data Set</a> used in <a href=https://ieeexplore.ieee.org/abstract/document/8452913>Verma & Rubin (2018)</a> with the model presented in <a href=http://papers.nips.cc/paper/6668-avoiding-discrimination-through-causal-reasoning>Kilbertus et al. (2017)</a>.

The first thing we try is analysing the data and try to find a useful causal model to apply the method.

### Imports

In [6]:
%matplotlib inline
import numpy as np
import pandas as pd
#import fairensics as fr
import matplotlib.pyplot as plt
import collections as col
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from aif360.datasets import GermanDataset
from aif360.metrics import ClassificationMetric
from fairensics.data.synthetic_dataset import SyntheticDataset
from fairensics.data.decision_boundary import DecisionBoundary 
from fairensics.methods import FairDisparateImpact, AccurateDisparateImpact
from fairensics.methods import DisparateMistreatment
from fairensics.methods import PreferentialFairness
from sklearn.linear_model import LogisticRegression

#  Hide warnings of LinearSVC, LogisticRegression
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

## Preprocessing the data

The data set has 20 attributes:

 - Checking account: 
     - A11: x < 0 DM
     - A12: 0 <= x < 200 DM
     - A13: x >= 200 DM
     - A14: no account
     
     
 - Credit duration in months
 
 
 - Credit history:
     - A30: no credits taken / all credits paid back duly
     - A31: all credits at this bank paid back duly
     - A32: existing credits paid back duly till now
     - A33: delay in paying off in the past
     - A34: critical account / other credits existing (not at this bank)
     
     
 - Purpose:
     - A40: car (new)
     - A41: car (used)
     - A42: furniture / equipment
     - A43: radio / television
     - A44: domestic appliance
     - A45: repairs
     - A46: education
     - A47: (vacation - but does not exist in the data set?)
     - A48: retraining
     - A49: business
     - A410: others
     
     
 - Credit amount
 
 
 - Savings account/bonds:
     - A61: x < 100 DM
     - A62: 100 <= x < 500 DM
     - A63: 500 <= x < 1000 DM
     - A64: x >= 1000 DM
     - A65: unknown / no savings account
     
     
 - Present employment since:
     - A71: unemployed
     - A72: x < 1 year
     - A73: 1 <= x < 4 years
     - A74: 4 <= x < 7 years
     - A75: x >= 7 years
     
     
 - Installment rate in percentage of disposable income
 
 
 - Personal status and sex:
     - A91: male, divorced / separated
     - A92: female, divorced / separated / married
     - A93: male, single
     - A94: male, married
     - A95: female, single
     
     
 - Other debtors / guarantors:
     - A101: none
     - A102: co-applicant
     - A103: guarantor
     
     
 - Present residence since (probably in years)
 
 
 - Property:
     - A121: real estate
     - A122: if not A121: building society savings agreement / life insurance
     - A123: if not A121 / 122: car or other, not in attribute 6
     - A124: unknown / no property
     
     
 - Age in years
 
 
 - Other installment plans:
     - A141: bank
     - A142: stores
     - A143: none
     
     
 - Housing:
     - A151: rent
     - A152: own
     - A153: for free
     
     
 - Number of existing credits at this bank
 
 
 - Job:
     - A171: unemployed / unskilled - non-resident
     - A172: unskilled - resident
     - A173: skilled employee / official
     - A174: management / self-employed / highly qualified employee / officer
     
     
 - Number of people being liable to provide maintenance for
 
 
 - Telephone:
     - A191: none
     - A192: yes, registered under the customers name
     
     
 - foreign worker:
     - A201: yes
     - A202: no
     
     
The last entry is the predicted credit score, which is the outcome we want:   
 - credit score:
     - 1: Good
     - 2: Bad

Installemnt rate: The actual term is, as per the data, "Installment rate in percentage of disposable income". This comes into picture if you are paying EMI (installment payments) for any loan, like house, car, TV/Electronic Goods, other loans. When a bank extends you loan, like home loan, it ensures that your total outgo on EMIs does not exceed certain percentage limit of your take home salary/income. This is done to prevent you from defaulting by taking larger loans that you can reasonably service. The exact percentage varies between countries. So all your EMIs together should not exceed this percentage.

For example, Alice is requesting a loan amount of 1567 DM for a duration of 12 months for the purpose of purchasing a television, with a positive checking account balance that is smaller than 200 DM, having less than 100 DM in savings account, and having one existing credit at this bank. She duly paid existing credits at the bank till now and has no other installment plan. She possesses a car and owns a house, has been living at the present residence for one year and has a registered telephone. She is a skilled employee, working in the present employment for past four years. She is a 22-year-old married female and is a German citizen. She has one dependent and no guarantors. The recorded outcome for Alice (attribute #21) is a good credit score.


In [85]:
attributes = ["checking account", "duration", "credit history", "purpose", "credit amount", "savings", 
              "employment (since)", "installment rate", "status and sex", "debtors/guarantors", "residence (since)",
             "property", "age", "other installment plans", "housing", "existing credits", "job", 
             "liable people", "phone", "foreign worker", "credit score"]

#alice = np.array([["A12", 12, "A32", "A43", 1567, "A61", "A74", 2, "A92", "A101", 1, "A121", 22, "A143", "A152", 1,
        # "A173", 1, "A192", "A202", 1]])

#alice_df = pd.read_table("data/german_credit/alice.data", sep="\s", names=attributes)
#alice_score = np.ravel(pd.DataFrame(alice_df["credit score"]))
#del alice_df["credit score"]

df = pd.read_table("data/german_credit/german.data", sep="\s", names=attributes)

# extracts the credit score into its own data frame and deletes it from the imported data frame.
score = np.ravel(pd.DataFrame(df["credit score"]))
del df["credit score"]

df.head(10)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,checking account,duration,credit history,purpose,credit amount,savings,employment (since),installment rate,status and sex,debtors/guarantors,residence (since),property,age,other installment plans,housing,existing credits,job,liable people,phone,foreign worker
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201
5,A14,36,A32,A46,9055,A65,A73,2,A93,A101,4,A124,35,A143,A153,1,A172,2,A192,A201
6,A14,24,A32,A42,2835,A63,A75,3,A93,A101,4,A122,53,A143,A152,1,A173,1,A191,A201
7,A12,36,A32,A41,6948,A61,A73,2,A93,A101,2,A123,35,A143,A151,1,A174,1,A192,A201
8,A14,12,A32,A43,3059,A64,A74,2,A91,A101,4,A121,61,A143,A152,1,A172,1,A191,A201
9,A12,30,A34,A40,5234,A61,A71,4,A94,A101,2,A123,28,A143,A152,2,A174,1,A191,A201


In [86]:
#alice_df.dtypes
df.tail()

,checking account,duration,credit history,purpose,credit amount,savings,employment (since),installment rate,status and sex,debtors/guarantors,residence (since),property,age,other installment plans,housing,existing credits,job,liable people,phone,foreign worker
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,4,A122,40,A143,A152,1,A174,1,A192,A201
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,4,A123,38,A143,A152,1,A173,1,A191,A201
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,4,A124,23,A143,A153,1,A173,1,A192,A201
999,A12,45,A34,A41,4576,A62,A71,3,A93,A101,4,A123,27,A143,A152,1,A173,1,A191,A201
1000,A12,12,A32,A43,1567,A61,A74,2,A92,A101,1,A121,22,A143,A152,1,A173,1,A192,A202


In [87]:
score[:10]

array([1, 2, 1, 1, 2, 1, 1, 1, 1, 2])

In [88]:
df.dtypes

checking account           object
duration                    int64
credit history             object
purpose                    object
credit amount               int64
savings                    object
employment (since)         object
installment rate            int64
status and sex             object
debtors/guarantors         object
residence (since)           int64
property                   object
age                         int64
other installment plans    object
housing                    object
existing credits            int64
job                        object
liable people               int64
phone                      object
foreign worker             object
dtype: object

In [89]:
col.Counter(df["status and sex"])

Counter({'A93': 548, 'A92': 311, 'A91': 50, 'A94': 92})

To be able to use the data set for classification, we have to convert all categorical attributes to a set of binary features as done in <a href=https://ieeexplore.ieee.org/abstract/document/8452913>Verma & Rubin (2018)</a> as well.

**Update: We do not do the female / male thing for now!**

Also, since there is no attribute `single female`, we throw out `single male`. And since divorced, separated and married females are taken together as one attribute we also see `divorced / separated male` and `married male` as one. This basically leaves us with a binary feature which is either `male` or `female`.

In [91]:
bin_df = df.copy()
bin_df.head(10)

,checking account,duration,credit history,purpose,credit amount,savings,employment (since),installment rate,status and sex,debtors/guarantors,residence (since),property,age,other installment plans,housing,existing credits,job,liable people,phone,foreign worker
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201
5,A14,36,A32,A46,9055,A65,A73,2,A93,A101,4,A124,35,A143,A153,1,A172,2,A192,A201
6,A14,24,A32,A42,2835,A63,A75,3,A93,A101,4,A122,53,A143,A152,1,A173,1,A191,A201
7,A12,36,A32,A41,6948,A61,A73,2,A93,A101,2,A123,35,A143,A151,1,A174,1,A192,A201
8,A14,12,A32,A43,3059,A64,A74,2,A91,A101,4,A121,61,A143,A152,1,A172,1,A191,A201
9,A12,30,A34,A40,5234,A61,A71,4,A94,A101,2,A123,28,A143,A152,2,A174,1,A191,A201


In [92]:
col.Counter(bin_df["status and sex"])

Counter({'A93': 548, 'A92': 311, 'A91': 50, 'A94': 92})

In [93]:
# male = 0, female = 1
status_map = {'A91': 0, 'A93': 0, 'A94': 0,
                  'A92': 1, 'A95': 1}
bin_df['sex'] = bin_df['status and sex'].replace(status_map).astype(int)
bin_df.drop("status and sex", inplace=True, axis=1)
bin_df.head(10)

,checking account,duration,credit history,purpose,credit amount,savings,employment (since),installment rate,debtors/guarantors,residence (since),property,age,other installment plans,housing,existing credits,job,liable people,phone,foreign worker,sex
0,A11,6,A34,A43,1169,A65,A75,4,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,0
1,A12,48,A32,A43,5951,A61,A73,2,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,1
2,A14,12,A34,A46,2096,A61,A74,2,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,0
3,A11,42,A32,A42,7882,A61,A74,2,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,0
4,A11,24,A33,A40,4870,A61,A73,3,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,0
5,A14,36,A32,A46,9055,A65,A73,2,A101,4,A124,35,A143,A153,1,A172,2,A192,A201,0
6,A14,24,A32,A42,2835,A63,A75,3,A101,4,A122,53,A143,A152,1,A173,1,A191,A201,0
7,A12,36,A32,A41,6948,A61,A73,2,A101,2,A123,35,A143,A151,1,A174,1,A192,A201,0
8,A14,12,A32,A43,3059,A64,A74,2,A101,4,A121,61,A143,A152,1,A172,1,A191,A201,0
9,A12,30,A34,A40,5234,A61,A71,4,A101,2,A123,28,A143,A152,2,A174,1,A191,A201,0


As we can see when counting the numbers again we did not change the amount of male or female individuals in the set, just combined the amounts of male and female individuals with different status.

In [94]:
col.Counter(bin_df["sex"])

Counter({0: 690, 1: 311})

In [95]:
bin_df.dtypes

checking account           object
duration                    int64
credit history             object
purpose                    object
credit amount               int64
savings                    object
employment (since)         object
installment rate            int64
debtors/guarantors         object
residence (since)           int64
property                   object
age                         int64
other installment plans    object
housing                    object
existing credits            int64
job                        object
liable people               int64
phone                      object
foreign worker             object
sex                         int64
dtype: object

In [96]:
#bin_df = bin_df.append(alice_df)
bin_df = pd.get_dummies(bin_df, prefix="", prefix_sep="")
bin_df.tail(10)

,duration,credit amount,installment rate,residence (since),age,existing credits,liable people,sex,A11,A12,...,A152,A153,A171,A172,A173,A174,A191,A192,A201,A202
991,15,1569,4,4,34,1,2,0,0,0,...,1,0,0,1,0,0,1,0,1,0
992,18,1936,2,4,23,2,1,0,1,0,...,0,0,0,1,0,0,1,0,1,0
993,36,3959,4,3,30,1,1,0,1,0,...,1,0,0,0,0,1,0,1,1,0
994,12,2390,4,3,50,1,1,0,0,0,...,1,0,0,0,1,0,0,1,1,0
995,12,1736,3,4,31,1,1,1,0,0,...,1,0,0,1,0,0,1,0,1,0
996,30,3857,4,4,40,1,1,0,1,0,...,1,0,0,0,0,1,0,1,1,0
997,12,804,4,4,38,1,1,0,0,0,...,1,0,0,0,1,0,1,0,1,0
998,45,1845,4,4,23,1,1,0,1,0,...,0,1,0,0,1,0,0,1,1,0
999,45,4576,3,4,27,1,1,0,0,1,...,1,0,0,0,1,0,1,0,1,0
1000,12,1567,2,1,22,1,1,1,0,1,...,1,0,0,0,1,0,0,1,0,1


In [97]:
bin_df.tail()

,duration,credit amount,installment rate,residence (since),age,existing credits,liable people,sex,A11,A12,...,A152,A153,A171,A172,A173,A174,A191,A192,A201,A202
996,30,3857,4,4,40,1,1,0,1,0,...,1,0,0,0,0,1,0,1,1,0
997,12,804,4,4,38,1,1,0,0,0,...,1,0,0,0,1,0,1,0,1,0
998,45,1845,4,4,23,1,1,0,1,0,...,0,1,0,0,1,0,0,1,1,0
999,45,4576,3,4,27,1,1,0,0,1,...,1,0,0,0,1,0,1,0,1,0
1000,12,1567,2,1,22,1,1,1,0,1,...,1,0,0,0,1,0,0,1,0,1


In [98]:
# not needed for this format - only if we really want to get it to the format of Verma & Rubin
# which seems not necessary

#del bin_df["A202"], bin_df["A192"], bin_df["A174"], bin_df["A153"], bin_df["A143"], bin_df["A124"]
#del bin_df["A103"], bin_df["A75"], bin_df["A65"], bin_df["A410"], bin_df["A34"], bin_df["A14"], bin_df["A94"]
#bin_df.head(10)

In [99]:
alice_df = bin_df.loc[len(bin_df.index) - 1, :]
alice_score = score[len(bin_df.index) - 1]
bin_df.drop(len(bin_df.index) - 1, inplace=True)
score = np.delete(score, len(bin_df.index) - 1, 0)
bin_df.tail()

,duration,credit amount,installment rate,residence (since),age,existing credits,liable people,sex,A11,A12,...,A152,A153,A171,A172,A173,A174,A191,A192,A201,A202
995,12,1736,3,4,31,1,1,1,0,0,...,1,0,0,1,0,0,1,0,1,0
996,30,3857,4,4,40,1,1,0,1,0,...,1,0,0,0,0,1,0,1,1,0
997,12,804,4,4,38,1,1,0,0,0,...,1,0,0,0,1,0,1,0,1,0
998,45,1845,4,4,23,1,1,0,1,0,...,0,1,0,0,1,0,0,1,1,0
999,45,4576,3,4,27,1,1,0,0,1,...,1,0,0,0,1,0,1,0,1,0


## Data Analysis

We first want to perform 10-fold cross validation on the data set to reproduce the examples from <a href=https://ieeexplore.ieee.org/abstract/document/8452913>Verma & Rubin (2018)</a>.

In [112]:
cv = KFold(n_splits=10)
estimator = LogisticRegression()
clf = GridSearchCV(estimator=estimator, param_grid={'C':np.logspace(-4, 2, 10)}, cv=cv)
#print(score.shape)
clf.fit(bin_df, score)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-04, 4.64158883e-04, 2.15443469e-03, 1.00000000e-02,
       4.64158883e-02, 2.15443469e-01, 1.00000000e+00, 4.64158883e+00,
       2.15443469e+01, 1.00000000e+02])})

In [113]:
best = clf.best_estimator_

In [114]:
np.set_printoptions(threshold=np.inf)

# works only with the "old" format 
#print(np.mean(best.predict_proba(bin_df.loc[bin_df['A92'] == 1]), axis=0))
#print(np.mean(best.predict_proba(bin_df.loc[bin_df['A91'] == 1]), axis=0))
#print(np.mean(best.predict_proba(bin_df.loc[(bin_df['A91'] == 0) & (bin_df['A92'] == 0) & (bin_df['A93'] == 0)]),
#              axis=0))

# works only with the "new" format 
print(np.mean(best.predict_proba(bin_df.loc[bin_df['sex'] == 1]), axis=0))
print(np.mean(best.predict_proba(bin_df.loc[bin_df['sex'] == 0]), axis=0))

[0.65432517 0.34567483]
[0.70283528 0.29716472]


##### What is our prediction for Alice?

In [115]:
print(np.mean(best.predict_proba(alice_df.values.reshape(1, -1)), axis=0))

[0.79501572 0.20498428]


With probability of 84%, Alice has a good credit score.